## Loop for Spatial Structure Parameter

In [1]:
using PyCall
using Distributions, Statistics, Distances
using DataFrames
using LightGraphs, SimpleWeightedGraphs
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = "/home/stephane/Science/cluster/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"
ocdir= "$wdir/oc"

cd(wdir)

┌ Info: Recompiling stale cache file /home/stephane/.julia/compiled/v1.2/GaiaClustering.ji for GaiaClustering [top-level]
└ @ Base loading.jl:1240


In [2]:
cd(ocdir)
files= glob("*csv")
cd(wdir)

ilabel= []

i= 0
for f in files
    i+= 1
    # oc= CSV.read("$ocdir/$f" , delim= ";")
    ocfile= "$ocdir/$f"
    println("## ocfile: $f")
    println("## fitting the radial profile....")
    
    ntest= 50 ; nbin=20 ; niter=50000
    θfit, θfiterr= spatialParameter(ocfile, ntest, nbin, niter, true)
    println(θfit)
    println(θfiterr)
    println(" ")
    push!(ilabel, i)
end

## ocfile: ASCC 10-2.7deg-oc.csv
## fitting the radial profile....
## file /home/stephane/Science/cluster/GAIA/products/oc/ASCC 10-2.7deg-oc.csv read...
## Density profiles computed...
## Prior Guess:
GaiaClustering.modelCauchy(4.737930965435087, 1.9269167756933834, 3.0)
## prior estimated...
## Running mcmc 2D fitting


MethodError: MethodError: no method matching theta(::GaiaClustering.mcmcCauchy, ::GaiaClustering.sc2dcentered, ::typeof(GaiaClustering.likelihood2dbin), ::Bool)